In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

trade = pd.read_csv('trade_train.csv', index_col=0)
stock = pd.read_csv('stocks.csv', index_col=0)
answer = pd.read_csv('answer_sheet.csv')

In [4]:
final_stock = stock.loc[stock['20년7월TOP3대상여부']=='Y'].reset_index(drop=True)
final_stock.drop(['종목명','20년7월TOP3대상여부'], axis=1, inplace=True)

final_stock['year'] = final_stock['기준일자'].map(lambda x : str(x)[:4])
final_stock['month'] = final_stock['기준일자'].map(lambda x : str(x)[4:6])
final_stock['day'] = final_stock['기준일자'].map(lambda x : str(x)[6:])

company_list = final_stock['종목번호'].unique()

def fourWeeks(x):
    if x < 8:
        return 1
    elif x < 15:
        return 2
    elif x < 22:
        return 3
    else:
        return 4

final_stock['week'] = final_stock['day'].astype(np.int16).map(fourWeeks)


In [5]:
final_stock['week'].value_counts()

4    10858
3     8347
2     8215
1     8063
Name: week, dtype: int64

In [6]:

weekby_df =  pd.DataFrame(final_stock.groupby(['year','month','종목번호','week'])['거래량'].agg(['mean', 'max', 'min', 'std']))\
    .unstack().reset_index()
weekby_df['기준일자'] = weekby_df['year'].map(str) + weekby_df['month'].map(str)
weekby_df.columns = ['year','month','종목번호','거래량1_mean','거래량2_mean','거래량3_mean','거래량4_mean','거래량1_max','거래량2_max','거래량3_max','거래량4_max','거래량1_min','거래량2_min','거래량3_min','거래량4_min','거래량1_std','거래량2_std','거래량3_std','거래량4_std','기준일자']

weekby_df['거래량_mean_weekdiff41'] = weekby_df['거래량4_mean'] - weekby_df['거래량1_mean']
weekby_df['거래량_mean_weekdiff42'] = weekby_df['거래량4_mean'] - weekby_df['거래량2_mean']
weekby_df['거래량_mean_weekdiff43'] = weekby_df['거래량4_mean'] - weekby_df['거래량3_mean']

weekby_df['거래량_max_weekdiff41'] = weekby_df['거래량4_max'] - weekby_df['거래량1_max']
weekby_df['거래량_max_weekdiff42'] = weekby_df['거래량4_max'] - weekby_df['거래량2_max']
weekby_df['거래량_max_weekdiff43'] = weekby_df['거래량4_max'] - weekby_df['거래량3_max']

weekby_df['거래량_min_weekdiff41'] = weekby_df['거래량4_min'] - weekby_df['거래량1_min']
weekby_df['거래량_min_weekdiff42'] = weekby_df['거래량4_min'] - weekby_df['거래량2_min']
weekby_df['거래량_min_weekdiff43'] = weekby_df['거래량4_min'] - weekby_df['거래량3_min']

fillna_func = lambda x : x.fillna(x.mean())
weekby_df = weekby_df.groupby(['종목번호','year']).apply(fillna_func)

weekby_df.index = range(weekby_df.shape[0])

weekby_df_drop_cols = ['year','month','거래량1_mean','거래량2_mean','거래량3_mean','거래량1_max','거래량2_max','거래량3_max','거래량1_min','거래량2_min','거래량3_min']
weekby_df.drop(weekby_df_drop_cols, inplace=True, axis=1)

print(weekby_df.shape)
weekby_df.head()

(1725, 18)


,종목번호,거래량4_mean,거래량4_max,거래량4_min,거래량1_std,거래량2_std,거래량3_std,거래량4_std,기준일자,거래량_mean_weekdiff41,거래량_mean_weekdiff42,거래량_mean_weekdiff43,거래량_max_weekdiff41,거래량_max_weekdiff42,거래량_max_weekdiff43,거래량_min_weekdiff41,거래량_min_weekdiff42,거래량_min_weekdiff43
0,A000100,29075.250000,50035.0,18997.0,136743.365328,5960.026804,6328.185577,10956.123529,201907,-126822.950000,-15499.750000,-12617.550000,-345722.0,-3097.0,-1081.0,-41327.0,-19358.0,-14482.0
1,A000100,34131.571429,51455.0,23481.0,16324.680233,11080.926351,3395.221790,9352.261328,201908,-25283.228571,3760.971429,9204.071429,-33553.0,3513.0,22457.0,-16334.0,6261.0,1990.0
2,A000100,25128.000000,33724.0,16450.0,3208.891366,52791.795158,23545.205336,5691.504933,201909,992.600000,-47574.000000,-27779.600000,6496.0,-96444.0,-55824.0,-2458.0,-9906.0,-19800.0
3,A000100,31877.750000,50118.0,23046.0,4851.435423,6209.414432,3443.524256,8533.890400,201910,1541.250000,633.000000,3497.350000,14461.0,13809.0,16565.0,-1157.0,470.0,-2632.0
4,A000100,31503.333333,65260.0,15802.0,29276.735889,9269.334712,7539.677798,17594.827634,201911,-11207.266667,-6142.666667,4837.733333,-29194.0,18794.0,30935.0,-8113.0,-8324.0,420.0


In [7]:
stock_df = pd.DataFrame(final_stock.groupby(['year','month','종목번호'])['거래금액_만원단위'].agg(['mean', 'max', 'min', 'std'])).reset_index()
stock_df['기준일자'] = stock_df['year'].map(str) + stock_df['month'].map(str)
stock_df.columns = ['year','month','종목번호','거래금액_mean','거래금액_max', '거래금액_min', '거래금액_std','기준일자']
stock_df.drop(['year','month'], inplace=True, axis=1)
print(stock_df.shape)
stock_df.head()


(1725, 6)


,종목번호,거래금액_mean,거래금액_max,거래금액_min,거래금액_std,기준일자
0,A000100,1.509332e+06,10139357,440174,2.002812e+06,201907
1,A000140,1.266896e+06,9976705,5792,2.479253e+06,201907
2,A000440,1.288330e+04,32739,2365,9.501437e+03,201907
3,A000660,2.244919e+07,38395774,10526521,7.441696e+06,201907
4,A000885,1.351755e+05,438430,35199,8.610133e+04,201907


In [8]:
stock_weekby_df = pd.merge(stock_df, weekby_df, how='left', on=['종목번호','기준일자'])

In [9]:
cols = ['종목번호','시장구분','표준산업구분코드_대분류','표준산업구분코드_중분류']
add_df =  final_stock[cols].drop_duplicates().reset_index(drop=True)
final_company = pd.merge(stock_weekby_df,add_df, on='종목번호')

final_company.rename({'기준일자' : '기준년월'}, inplace=True, axis=1)
final_company['기준년월'] = final_company['기준년월'].astype(np.int32)

final_company.columns

Index(['종목번호', '거래금액_mean', '거래금액_max', '거래금액_min', '거래금액_std', '기준년월',
       '거래량4_mean', '거래량4_max', '거래량4_min', '거래량1_std', '거래량2_std', '거래량3_std',
       '거래량4_std', '거래량_mean_weekdiff41', '거래량_mean_weekdiff42',
       '거래량_mean_weekdiff43', '거래량_max_weekdiff41', '거래량_max_weekdiff42',
       '거래량_max_weekdiff43', '거래량_min_weekdiff41', '거래량_min_weekdiff42',
       '거래량_min_weekdiff43', '시장구분', '표준산업구분코드_대분류', '표준산업구분코드_중분류'],
      dtype='object')

Index(['종목번호', '거래금액_mean', '거래금액_max', '거래금액_min', '거래금액_std', '기준년월',
       '거래량4_mean', '거래량4_max', '거래량4_min', '거래량1_std', '거래량2_std', '거래량3_std',
       '거래량4_std', '거래량_mean_weekdiff41', '거래량_mean_weekdiff42',
       '거래량_mean_weekdiff43', '거래량_max_weekdiff41', '거래량_max_weekdiff42',
       '거래량_max_weekdiff43', '거래량_min_weekdiff41', '거래량_min_weekdiff42',
       '거래량_min_weekdiff43', '시장구분', '표준산업구분코드_대분류', '표준산업구분코드_중분류'],
      dtype='object')

In [10]:
com_data = final_stock.groupby(['year','month','종목번호'])['종목고가','종목저가'].agg(['mean', 'max', 'min', 'std']).reset_index()
print(com_data.shape)
com_data['기준년월'] = com_data['year'] + com_data['month']
com_data.drop(['year','month'], axis=1, inplace=True)
com_data['기준년월'] = com_data['기준년월'].astype(np.int32)
com_data.columns = ['종목번호','종목고가_mean','종목고가_max','종목고가_min','종목고가_std','종목저가_mean','종목저가_max','종목저가_min','종목저가_std', '기준년월']
com_data.head()

(1725, 11)


,종목번호,종목고가_mean,종목고가_max,종목고가_min,종목고가_std,종목저가_mean,종목저가_max,종목저가_min,종목저가_std,기준년월
0,A000100,235717.391304,267500,220000,11622.691449,229500.000000,250000,213500,9452.032009,201907
1,A000140,10545.217391,14150,9160,1199.408451,9701.304348,12000,8910,659.946998,201907
2,A000440,6826.086957,7540,6130,411.446883,6579.565217,7190,5760,406.486925,201907
3,A000660,75608.695652,80700,68000,3934.799642,73443.478261,78200,66200,3765.733791,201907
4,A000885,33491.304348,36200,31500,1483.672668,31717.391304,33900,29800,1156.071787,201907


In [11]:
stockForSeq2Seq = pd.merge(com_data,final_company, on=['종목번호','기준년월'], how='left')

In [12]:
stockForSeq2Seq.to_csv('stockForSeq2Seq.csv', index=False)

# trade for seq2seq

In [13]:
# stockForSeq2Seq = pd.read_csv('stockForSeq2Seq.csv')

In [14]:
top3_trade = trade.loc[trade['종목번호'].isin(company_list)].reset_index(drop=True)
top3_trade.columns

Index(['기준년월', '그룹번호', '그룹내고객수', '종목번호', '그룹내_매수여부', '그룹내_매도여부', '매수고객수',
       '매도고객수', '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값'],
      dtype='object')

In [15]:
# strange value
top3_trade.iloc[348,10] = \
    top3_trade.loc[(top3_trade['그룹번호']=='MAD48')&(top3_trade['종목번호']=='A035420')][1:]['매수가격_중앙값'].mean()

In [16]:
cols = ['기준년월','그룹번호','종목번호','매수고객수','매도고객수']
final_trade = top3_trade[cols]

In [17]:
# 각 종목마다 그룹 만들기
tmp_com = company_list
for i in range(0,47):
    tmp_com = np.append(tmp_com,company_list)

tmp_group = sorted(final_trade['그룹번호'].unique())
for i in range(0,134):
    tmp_group = np.append(tmp_group, sorted(final_trade['그룹번호'].unique()))

tmp_df = pd.DataFrame(tmp_com).sort_values(by=0).reset_index(drop=True)

tmp_df['그룹번호'] = tmp_group
tmp_df.columns = ['종목번호','그룹번호']

# 
mse10 = ['MAD01','MAD04','MAD09','MAD13','MAD14','MAD17','MAD18','MAD21','MAD38','MAD42','MAD44']
mse20 = ['MAD05','MAD06','MAD23','MAD35','MAD41','MAD43','MAD46']
tmp_df['group_mse'] = np.where(tmp_df['그룹번호'].isin(mse10), 2, 1)
tmp_df['group_mse'] = np.where(tmp_df['그룹번호'].isin(mse20), 3, tmp_df['group_mse'])

tmp_df.tail()

,종목번호,그룹번호,group_mse
6475,A353810,MAD44,2
6476,A353810,MAD45,1
6477,A353810,MAD46,3
6478,A353810,MAD47,1
6479,A353810,MAD48,1


In [18]:
seq2seq_df = pd.merge(stockForSeq2Seq, tmp_df, on='종목번호', how='left')
print(seq2seq_df.shape)

seq2seq_df.head()

(82800, 35)


,종목번호,종목고가_mean,종목고가_max,종목고가_min,종목고가_std,종목저가_mean,종목저가_max,종목저가_min,종목저가_std,기준년월,...,거래량_max_weekdiff42,거래량_max_weekdiff43,거래량_min_weekdiff41,거래량_min_weekdiff42,거래량_min_weekdiff43,시장구분,표준산업구분코드_대분류,표준산업구분코드_중분류,그룹번호,group_mse
0,A000100,235717.391304,267500,220000,11622.691449,229500.0,250000,213500,9452.032009,201907,...,-3097.0,-1081.0,-41327.0,-19358.0,-14482.0,코스피,제조업,의료용 물질 및 의약품 제조업,MAD01,2
1,A000100,235717.391304,267500,220000,11622.691449,229500.0,250000,213500,9452.032009,201907,...,-3097.0,-1081.0,-41327.0,-19358.0,-14482.0,코스피,제조업,의료용 물질 및 의약품 제조업,MAD02,1
2,A000100,235717.391304,267500,220000,11622.691449,229500.0,250000,213500,9452.032009,201907,...,-3097.0,-1081.0,-41327.0,-19358.0,-14482.0,코스피,제조업,의료용 물질 및 의약품 제조업,MAD03,1
3,A000100,235717.391304,267500,220000,11622.691449,229500.0,250000,213500,9452.032009,201907,...,-3097.0,-1081.0,-41327.0,-19358.0,-14482.0,코스피,제조업,의료용 물질 및 의약품 제조업,MAD04,2
4,A000100,235717.391304,267500,220000,11622.691449,229500.0,250000,213500,9452.032009,201907,...,-3097.0,-1081.0,-41327.0,-19358.0,-14482.0,코스피,제조업,의료용 물질 및 의약품 제조업,MAD05,3


In [19]:
tmp_numGroup = top3_trade[['그룹번호','그룹내고객수']].drop_duplicates(subset=['그룹번호']).reset_index(drop=True)
print(tmp_numGroup.shape)

(48, 2)


In [20]:
seq2seq_df = seq2seq_df.merge(tmp_numGroup, on='그룹번호', how='left')
seq2seq_df = seq2seq_df.merge(final_trade, on=['그룹번호','종목번호','기준년월'], how='left')

print(seq2seq_df.shape)

(82800, 38)


In [21]:
seq2seq_df.isnull().sum()

종목번호                       0
종목고가_mean                  0
종목고가_max                   0
종목고가_min                   0
종목고가_std                   0
종목저가_mean                  0
종목저가_max                   0
종목저가_min                   0
종목저가_std                   0
기준년월                       0
거래금액_mean                  0
거래금액_max                   0
거래금액_min                   0
거래금액_std                   0
거래량4_mean                  0
거래량4_max                   0
거래량4_min                   0
거래량1_std                  48
거래량2_std                  48
거래량3_std                   0
거래량4_std                   0
거래량_mean_weekdiff41       48
거래량_mean_weekdiff42       48
거래량_mean_weekdiff43        0
거래량_max_weekdiff41        48
거래량_max_weekdiff42        48
거래량_max_weekdiff43         0
거래량_min_weekdiff41        48
거래량_min_weekdiff42        48
거래량_min_weekdiff43         0
시장구분                       0
표준산업구분코드_대분류               0
표준산업구분코드_중분류               0
그룹번호                       0
group_mse     

In [22]:
# NA
seq2seq_df['매수고객수'].fillna(0, inplace=True)
seq2seq_df['매도고객수'].fillna(0, inplace=True)

fillna_func = lambda x : x.fillna(x.mean())
seq2seq_df = seq2seq_df.groupby(['종목번호']).apply(fillna_func)

In [23]:
seq2seq_df.index = range(seq2seq_df.shape[0])
seq2seq_df.to_csv('seq2seq_df.csv', index=False)

In [24]:
print(seq2seq_df.shape)
seq2seq_df.isnull().sum()

(82800, 38)


종목번호                   0
종목고가_mean              0
종목고가_max               0
종목고가_min               0
종목고가_std               0
종목저가_mean              0
종목저가_max               0
종목저가_min               0
종목저가_std               0
기준년월                   0
거래금액_mean              0
거래금액_max               0
거래금액_min               0
거래금액_std               0
거래량4_mean              0
거래량4_max               0
거래량4_min               0
거래량1_std               0
거래량2_std               0
거래량3_std               0
거래량4_std               0
거래량_mean_weekdiff41    0
거래량_mean_weekdiff42    0
거래량_mean_weekdiff43    0
거래량_max_weekdiff41     0
거래량_max_weekdiff42     0
거래량_max_weekdiff43     0
거래량_min_weekdiff41     0
거래량_min_weekdiff42     0
거래량_min_weekdiff43     0
시장구분                   0
표준산업구분코드_대분류           0
표준산업구분코드_중분류           0
그룹번호                   0
group_mse              0
그룹내고객수                 0
매수고객수                  0
매도고객수                  0
dtype: int64

In [25]:
tmp = seq2seq_df.loc[seq2seq_df['매수고객수']!=0].reset_index(drop=True)
tmp['target'] = tmp.groupby(['종목번호','그룹번호'])['매수고객수'].shift(-1).reset_index(drop=True)
tmp.dropna(inplace=True)
for i in ['종목고가_mean', '종목고가_max', '종목고가_min', '종목고가_std', '종목저가_mean',
       '종목저가_max', '종목저가_min', '종목저가_std', '기준년월', '거래금액_mean', '거래금액_max',
       '거래금액_min', '거래금액_std', '거래량4_mean', '거래량4_max', '거래량4_min', '거래량1_std',
       '거래량2_std', '거래량3_std', '거래량4_std', '거래량_mean_weekdiff41',
       '거래량_mean_weekdiff42', '거래량_mean_weekdiff43', '거래량_max_weekdiff41',
       '거래량_max_weekdiff42', '거래량_max_weekdiff43', '거래량_min_weekdiff41',
       '거래량_min_weekdiff42', '거래량_min_weekdiff43', '그룹내고객수', '매수고객수', '매도고객수']:
    corr = np.corrcoef(tmp['target'], tmp[i])[0,1]
    print(f'target corr with : {i}, {corr}')

target corr with : 종목고가_mean, 0.03220906421424406
target corr with : 종목고가_max, 0.0323747160970236
target corr with : 종목고가_min, 0.030532398093261505
target corr with : 종목고가_std, 0.03364786949843165
target corr with : 종목저가_mean, 0.032508672788435465
target corr with : 종목저가_max, 0.03321859831948532
target corr with : 종목저가_min, 0.03110993862166436
target corr with : 종목저가_std, 0.03209646365585894
target corr with : 기준년월, 0.1373482259816295
target corr with : 거래금액_mean, 0.3355921338927938
target corr with : 거래금액_max, 0.2542088214092789
target corr with : 거래금액_min, 0.3467070984454042
target corr with : 거래금액_std, 0.21503193210120486
target corr with : 거래량4_mean, 0.18388661985408752
target corr with : 거래량4_max, 0.08881093682461996
target corr with : 거래량4_min, 0.29845004085393945
target corr with : 거래량1_std, 0.032158166343337166
target corr with : 거래량2_std, 0.02124438615365513
target corr with : 거래량3_std, -0.001448720046250285
target corr with : 거래량4_std, 0.007578531815821299
target corr with : 